In [22]:
import joblib
import pandas as pd
import json
import tweepy as tw
import configparser
import numpy as np
import re
import nltk

from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

#Conexão com o MongoDb
from pymongo import MongoClient

## Importando modelo pkl treinado

In [2]:
model = joblib.load('model.pkl')

In [4]:
model.steps

[('counts', CountVectorizer()), ('classifier', SVC(kernel='linear'))]

## Conectando ao MongoDB e importando dados #Eleicoes2022

In [5]:
config = configparser.ConfigParser()
config.read("config.ini")

['config.ini']

In [6]:
#Buscando chave de acesso 
mongo = config["mongodb"]["mongo"]

In [7]:
#conectando ao mongoDB
client_db = MongoClient(mongo)

In [8]:
#Conectando ao banco de dados eleicoes2022
db = client_db.get_database("eleicoes2022_db")

In [9]:
#CONECTANDO AO COLLECTION NAME TWEETS
collection_tweets = db.tweets_

In [15]:
df = pd.DataFrame(list(collection_tweets.find()))

In [16]:
df.head()

,_id,created_at,text,truncated,metadata,source,retweet_count,retweeted,lang
0,637786f0f88d752f97e1d28c,Thu Nov 17 14:42:43 +0000 2022,"In Brazil And U.S., Elections As Stress Tests For Democracy#politics #brazil #UnitedStates #democracy #eleições…",True,"{'iso_language_code': 'en', 'result_type': 'recent'}","<a href=""https://mobile.twitter.com"" rel=""nofollow"">Twitter Web App</a>",0,False,en
1,637786f0f88d752f97e1d28d,Thu Nov 17 14:42:42 +0000 2022,Continuamos na luta! #brazilwasstollen #Eleicoes2022,False,"{'iso_language_code': 'pt', 'result_type': 'recent'}","<a href=""http://twitter.com/download/android"" rel=""nofollow"">Twitter for Android</a>",0,False,pt
2,637786f0f88d752f97e1d28e,Thu Nov 17 14:36:46 +0000 2022,Veja a lista de 43 pessoas físicas e jurídicas suspeitas de financiar atos antidemocráticos (Piada pronta):.09 -…,True,"{'iso_language_code': 'pt', 'result_type': 'recent'}","<a href=""http://twitter.com/download/android"" rel=""nofollow"">Twitter for Android</a>",0,False,pt
3,637786f0f88d752f97e1d28f,Thu Nov 17 14:36:27 +0000 2022,É a política no Brasil tá como? #Politica #Brasil #Eleicoes2022 @iRodolffo te explica melhor 😂😂😂🧨🔫🖋️🔪💸🚜🥊🏹🍕🥩🍳🐀🐄,False,"{'iso_language_code': 'pt', 'result_type': 'recent'}","<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",0,False,pt
4,637786f0f88d752f97e1d290,Thu Nov 17 14:25:43 +0000 2022,@Seondetails #PERDEUMANÉ#LulaJá #LulaPresidente #Lula2022 #Lula#Eleições2022 #Eleiçoes22 #Brasil#FORABOLSONARO…,True,"{'iso_language_code': 'und', 'result_type': 'recent'}","<a href=""https://mobile.twitter.com"" rel=""nofollow"">Twitter Web App</a>",0,False,und


temos o intuido te testar nosso modelo, então não iremos nos preocupar no tratamento e criação de novas features

In [29]:
pd.options.display.max_colwidth = 130
df["text"]

0          In Brazil And U.S., Elections As Stress Tests For Democracy#politics #brazil #UnitedStates #democracy #eleições… 
1                                                                      Continuamos na luta! #brazilwasstollen #Eleicoes2022 
2         Veja a lista de 43 pessoas físicas e jurídicas suspeitas de financiar atos antidemocráticos (Piada pronta):.09 -… 
3            É a política no Brasil tá como? #Politica #Brasil #Eleicoes2022 @iRodolffo te explica melhor 😂😂😂🧨🔫🖋️🔪💸🚜🥊🏹🍕🥩🍳🐀🐄 
4           @Seondetails #PERDEUMANÉ#LulaJá #LulaPresidente #Lula2022 #Lula#Eleições2022 #Eleiçoes22 #Brasil#FORABOLSONARO… 
                                                               ...                                                          
799    O Partido PL descobriu muita coisa para pedir a anulação das eleições com 8 Senadores, os 99 Deputados Federais, 12… 
800      @SenadoFederal@camaradeputados @ArthurLira_@augustosnunes @EduGiraoOficial @folha_politica @EstadaoPolitica #BDRJ… 


definindo processo de tratamento dos tweets

In [23]:
def Preprocessing(instancia):
    stemmer = nltk.stem.RSLPStemmer()
    instancia = re.sub(r"http\S+", "", instancia).lower().replace('.','').replace(';','').replace('-','').replace(':','').replace(')','')
    stopwords = set(nltk.corpus.stopwords.words('portuguese'))
    palavras = [stemmer.stem(i) for i in instancia.split() if not i in stopwords]
    return (" ".join(palavras))

In [31]:
tweets = df["text"]

In [32]:
#processando os tweets
tweets_processados = [Preprocessing(i) for i in tweets]

In [26]:
tweets_processados

['in brazil and us, electiom str test democracy#politic #brazil #unitedstat #democracy #eleições…',
 'continu luta! #brazilwasstollen #eleicoes2022',
 'vej list 43 pess físic juríd suspeit financ ato antidemocr (pi pronta09 …',
 'polít brasil tá como? #pol #brasil #eleicoes2022 @irodolff explic melhor 😂😂😂🧨🔫🖋️🔪💸🚜🥊🏹🍕🥩🍳🐀🐄',
 '@seondetail #perdeumané#lulaj #lulapresid #lula2022 #lula#eleições2022 #eleiçoes22 #brasil#forabolsonaro…',
 '🤖 cot minist justiça, din defend revog decret bolson sobr armas#lul #eleicoes2022',
 '2022 rodrig garc (@rodrigogarcia_ #tarcis freit (@tarcisiogdf reún hoj 15h (ℹ…',
 '#eleiçõessbrasil2022 #eleições2022 #brasilnasru #brasilfoiroub #brasilacimadetudodeusacimadetod #',
 'segund dados, 26% brasil diz peg ranç camiset amarel #brasil #eleicoes2022…',
 'pronunc contr manifest golp esp apo govern #lul 30, qual…',
 'pé barr faç digo, faç canalhas@generalmo #eleicoes2022 #eleicoesnom #foraboz',
 'liberdad acim tudo?is pi mund result equaç milicoxpastorxlunático@stf…'

In [35]:
predicao = []
for tweet, pred in zip(tweets, model.predict(tweets_processados)):
    print (tweet, ", ", pred)
    predicao.append(pred)


In Brazil And U.S., Elections As Stress Tests For Democracy#politics #brazil #UnitedStates #democracy #eleições…  ,  Neutro
Continuamos na luta! #brazilwasstollen #Eleicoes2022  ,  Neutro
Veja a lista de 43 pessoas físicas e jurídicas suspeitas de financiar atos antidemocráticos (Piada pronta):.09 -…  ,  Neutro
É a política no Brasil tá como? #Politica #Brasil #Eleicoes2022 @iRodolffo te explica melhor 😂😂😂🧨🔫🖋️🔪💸🚜🥊🏹🍕🥩🍳🐀🐄  ,  Neutro
@Seondetails #PERDEUMANÉ#LulaJá #LulaPresidente #Lula2022 #Lula#Eleições2022 #Eleiçoes22 #Brasil#FORABOLSONARO…  ,  Neutro
🤖 Cotado para Ministério da Justiça, Dino defende revogação de decretos de Bolsonaro sobre armas#Lula #Eleicoes2022 ,  Neutro
2022 - Rodrigo Garcia (@rodrigogarcia_) e #Tarcisio de Freitas (@tarcisiogdf) se reúnem hoje às 15hs (ℹ…  ,  Neutro
#eleiçõessbrasil2022 #eleições2022 #BrasilNasRuas #BrasilFoiRoubado #BrasilAcimaDeTudoDeusAcimaDeTodos #  ,  Neutro
Segundo dados, 26% dos brasileiros dizem que pegaram ranço da camiseta amarela do #B

In [43]:
tweets_predicao = pd.DataFrame({"Tweets":tweets, "Classificacao":predicao})
                            
tweets_predicao.head()

,Tweets,Classificacao
0,"In Brazil And U.S., Elections As Stress Tests For Democracy#politics #brazil #UnitedStates #democracy #eleições…",Neutro
1,Continuamos na luta! #brazilwasstollen #Eleicoes2022,Neutro
2,Veja a lista de 43 pessoas físicas e jurídicas suspeitas de financiar atos antidemocráticos (Piada pronta):.09 -…,Neutro
3,É a política no Brasil tá como? #Politica #Brasil #Eleicoes2022 @iRodolffo te explica melhor 😂😂😂🧨🔫🖋️🔪💸🚜🥊🏹🍕🥩🍳🐀🐄,Neutro
4,@Seondetails #PERDEUMANÉ#LulaJá #LulaPresidente #Lula2022 #Lula#Eleições2022 #Eleiçoes22 #Brasil#FORABOLSONARO…,Neutro


In [46]:
import plotly.express as px
fig = px.bar(x = tweets_predicao["Classificacao"].value_counts().index, 
             y = tweets_predicao["Classificacao"].value_counts().values,
             title = "Classificação dos tweets com a #ELeicao2022")
fig.show()